In [1]:
from typing import Annotated, TypedDict

from langchain_core.messages import HumanMessage
from langchain_ollama import ChatOllama
from langgraph.graph import StateGraph, START, END, add_messages
from langgraph.checkpoint.memory import MemorySaver


class State(TypedDict):
    messages: Annotated[list, add_messages]


builder = StateGraph(State)
model = ChatOllama(model="gemma3:1b")


def chatbot(state: State):
    answer = model.invoke(state["messages"])
    return {"messages": [answer]}


builder.add_node("chatbot", chatbot)
builder.add_edge(START, "chatbot")
builder.add_edge("chatbot", END)

graph = builder.compile(checkpointer=MemorySaver())

/home/anseh/.pyenv/versions/langchain/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
thread1 = {"configurable": {"thread_id": "1"}}

result_1 = graph.invoke(
    {"messages": [HumanMessage("안녕하세요 저는 동원입니다.")]}, thread1
)
result_2 = graph.invoke({"messages": [HumanMessage("제 이름이 뭐죠?")]}, thread1)